In [1]:
import numpy as np
import pandas as pd

In [2]:
import string, re, unidecode

import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

from collections import Counter

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics.pairwise import cosine_similarity

# Read in

In [3]:
df_x = pd.read_csv("Data/dfx_nb0.csv", index_col = 0)
print(df_x.shape)

(172295, 25)


In [4]:
df_x['text'] = df_x['text'].fillna('')
df_x['textlower'] = df_x['text'].str.lower()
df_x['textlower'] = df_x['textlower'].str.replace('\n', ' ') # Replace all newlines

In [5]:
# Just for bottom print
df_x['title'] = df_x['title'].fillna('')

# Text processing

In [6]:
stemmer = SnowballStemmer('spanish')
remove_punc = str.maketrans('', '', string.punctuation)
stopwords_ascii = [unidecode.unidecode(w) for w in stopwords.words('spanish')]

def tokenize(s):
    s = unidecode.unidecode(s)
    s = s.translate(remove_punc)
    tokens = nltk.word_tokenize(s)
    filtered = [w for w in tokens if w not in stopwords_ascii]
    return [stemmer.stem(w) for w in filtered]

In [7]:
def dummy(x):
    return x

vectorizer = TfidfVectorizer(analyzer='word', tokenizer=dummy, preprocessor=dummy,
                            token_pattern=None)

In [8]:
df_t = df_x[df_x['textlower'].str.len() > 0]
print(df_t.shape)

(101797, 26)


## Tokenize

In [9]:
df_t.loc[:,'token'] = df_t.loc[:,'textlower'].apply(tokenize)

//anaconda3/envs/vzwa/lib/python3.7/site-packages/pandas/core/indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
//anaconda3/envs/vzwa/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [10]:
# Tokenize across all groups
def aggTokens(df):
    return df['token'].sum()

df_tgroup = df_t.groupby('uid').apply(aggTokens)

In [11]:
groups = sorted(df_t.uid.unique())
groups_titles = [df_x[df_x['uid'] == g]['title'].unique() for g in groups]

In [12]:
X = vectorizer.fit_transform(df_tgroup)
c = cosine_similarity(X)

In [13]:
def groupMsg(i):
    return df_x[df_x['uid'] == groups[i]]

def groupLatestDate(i):
    return groupMsg(i).sort_values('message_dt').iloc[-1]['message_dt']

trueCols = ['event', 'text', 'emojis',
       'audio_length', 'image', 'reply_to', 'reply_text', 'reply_image',
       'reply_thumb', 'reply_emojis', 'reply_type_audio', 'reply_type_image',
       'message_deleted', 'video_length', 'video_thumb', 'forwarded',
       'forwarded_highly', 'tel', 'message_dt', 'message_date', 'textlower']

## Process similarities

Match if one group contains 60\% of the other's messages (full inner merge)

In [14]:
rename_from_to = []

for i in zip(np.nonzero(c > 0.5)[0], np.nonzero(c > 0.5)[1]):
    if (i[0] >= i[1]):
        continue
    print(c[i[0], i[1]])
    print(i)
    print(groups[i[0]] + " / " + groups[i[1]])
    print(groups_titles[i[0]])
    print(groups_titles[i[1]])
    merge_size = pd.merge(groupMsg(i[0])[trueCols], groupMsg(i[1])[trueCols], how='inner').shape[0]
    
    if merge_size > 0.6 * min(groupMsg(i[0])[trueCols].shape[0], groupMsg(i[1])[trueCols].shape[0]) \
        and not (('-' in groups[i[0]]) and ('-' in groups[i[1]])):
        print("\nMERGE")
        
        if '-' in groups[i[0]]:
            rename_from_to.append((groups[i[1]], groups[i[0]]))
        elif '-' in groups[i[1]]:
            rename_from_to.append((groups[i[0]], groups[i[1]]))
        elif groupLatestDate(i[0]) >= groupLatestDate(i[1]):
            rename_from_to.append((groups[i[1]], groups[i[0]]))
        else:
            rename_from_to.append((groups[i[0]], groups[i[1]]))
    
    else:
        print()
        print(merge_size / min(groupMsg(i[0])[trueCols].shape[0], groupMsg(i[1])[trueCols].shape[0]))
        
    print()

0.5410617395392494
(1, 10)
07d5068cc56fa32bb22935edee3cf10a / 4057b3bafd4c6615ab55d292b8ca75f5
['VENEZOLANOS UNIDOS']
['VENEZOLANOS UNIDOS']

MERGE

0.5352425151217718
(2, 90)
0c069c3f9d40b1ab48639a0c0686dfc3 / 573185484739-1578495272
['VIAJES LATINOAMERICA 2']
['VIAJES LATINOAMERICA 2' 'VIAJES LATINOAMERICA 2…']

MERGE

1.0
(2, 162)
0c069c3f9d40b1ab48639a0c0686dfc3 / 5fec47ed4dc43982ac4e647aaecc48a2
['VIAJES LATINOAMERICA 2']
['VIAJES LATINOAMERICA 2']

MERGE

0.9483070617207925
(4, 6)
2a9efd099c8e26e3a8e189e617aa0e4a / 35c0a8c5a334567e7087db5c6c8d38c5
['COSTEÑOS CASEROS Y MAS']
['COSTEÑOS CASEROS Y MAS']

MERGE

0.979125371764508
(9, 149)
39c1800c883eff2c915903f30d38210d / 584245555958-1567221961
['Padres en Cúcuta']
['Show nay']

MERGE

0.9106251233706617
(11, 104)
43b1769e9012f53d9429819315f97f1f / 573212644564-1549511812
['StarCambioS']
['StarCambioS']

MERGE

1.0
(11, 167)
43b1769e9012f53d9429819315f97f1f / 85700215ac095301c08be263a8e96d4a
['StarCambioS']
['StarCambioS']

MERGE




0.0

0.5725794587866898
(21, 53)
51974115898-1579148636 / 573116409370-1578664391
['AmorVenezuel@']
['videos & demencias']

0.0

0.5286263323707939
(21, 74)
51974115898-1579148636 / 573135714229-1525652805
['AmorVenezuel@']
['VENTAS - EMPLEO - ARRIEDO' 'COMUNIDAD SW CALI']

0.0

0.5482196854357848
(21, 75)
51974115898-1579148636 / 573135852009-1585347056
['AmorVenezuel@']
['Lakrasy fresasde vnzla' 'Hora hott' 'Lakrasy fresas de vnzl'
 '.Supreme 5.0']

0.0

0.6615234977715284
(21, 81)
51974115898-1579148636 / 573145114610-1573649691
['AmorVenezuel@']
['Fotos y videos xxx' 'GRUPO  DE DESPARCHE…' 'GRUPO  DE DESPARCHE']

0.0

0.5342052121773815
(21, 98)
51974115898-1579148636 / 573204896999-1557775885
['AmorVenezuel@']
['>AMIGS SIN FRNTERAS<' '>AMIGS SIN FRNTERAS<…']

0.0

0.6106172685127569
(21, 101)
51974115898-1579148636 / 573209056220-1582183315
['AmorVenezuel@']
['SalsaBaul ineditas']

0.0

0.5715551769251878
(21, 154)
51974115898-1579148636 / 584247393733-1565574033
['AmorVenezuel@'


0.0

0.520517994433519
(30, 154)
573006401779-1563676942 / 584247393733-1565574033
['EMPLEO VENEZOLANOS' 'EMPLEO VENEZOLANOS…' 'Venezolanos unidos'
 'EN CUARENTENA']
['GENTE CHEVERE']

0.0

0.5004819142682124
(30, 161)
573006401779-1563676942 / 584264954187-1584036417
['EMPLEO VENEZOLANOS' 'EMPLEO VENEZOLANOS…' 'Venezolanos unidos'
 'EN CUARENTENA']
['Salsa baul.']

0.0

0.5901236230009873
(33, 56)
573006632902-1573570307 / 573118275688-1574895740
['Buenas amistades']
['XXX CASEROS Y MAS']

0.0

0.5231566604226654
(33, 81)
573006632902-1573570307 / 573145114610-1573649691
['Buenas amistades']
['Fotos y videos xxx' 'GRUPO  DE DESPARCHE…' 'GRUPO  DE DESPARCHE']

0.0

0.5795481801576345
(33, 120)
573006632902-1573570307 / 573505077552-1573850475
['Buenas amistades']
['Venezuela']

0.0

0.6371899976843463
(37, 71)
573014875618-1568166107 / 573132354041-1558060453
['En busca del amor']
['El Renacimiento']

0.0

1.0
(37, 168)
573014875618-1568166107 / 85b07927cac67371a7fad783185e3c98
['En b


0.0

0.5156958491908149
(45, 131)
573106687281-1572810835 / 584144043691-1548595172
['CONFORT SALUD']
['*NOTICIAS VZLA *']

0.0

0.5229477042786147
(48, 51)
573107070227-1558586309 / 573115240918-1554687011
['Ahizer tours']
['Agencia Miraflores']

0.0

0.5472663822015532
(48, 58)
573107070227-1558586309 / 573118547343-1545866247
['Ahizer tours']
['ServicioViajeros JP &']

0.0

0.5566580480121245
(48, 90)
573107070227-1558586309 / 573185484739-1578495272
['Ahizer tours']
['VIAJES LATINOAMERICA 2' 'VIAJES LATINOAMERICA 2…']

0.0

0.5825021935788697
(48, 121)
573107070227-1558586309 / 573507846957-1571104076
['Ahizer tours']
['Viajando por america' 'Viajando por america…' 'viaja con yoeslin']

0.0

0.5564428880764904
(49, 78)
573108189461-1574524235 / 573138303724-1574641574
['Venezolanos unidos en paz' 'Chamos' 'Venezolanos en Colombia']
['Los Panas de Medellín …' 'Los Panas de Medellín']

0.0

0.5874855489559881
(51, 58)
573115240918-1554687011 / 573118547343-1545866247
['Agencia Miraf

0.0

0.6371899976843463
(71, 174)
573132354041-1558060453 / 98e09c58f6c8dadb5a0718470da8e78a
['El Renacimiento']
['En busca del amor']

0.0

0.6153863571271405
(71, 178)
573132354041-1558060453 / bb5f0162ea905c338130adedd4e0b703
['El Renacimiento']
['Rumbas en Cali YAYI']

0.0

0.6371899976843463
(71, 182)
573132354041-1558060453 / c5b60e71e9324a17027931bab4014e86
['El Renacimiento']
['En busca del amor']

0.0

0.5589602125950446
(73, 78)
573134508951-1549727034 / 573138303724-1574641574
['Tv Libre Noticias']
['Los Panas de Medellín …' 'Los Panas de Medellín']

0.0

0.6460319565538348
(73, 118)
573134508951-1549727034 / 573227727724-1572791992
['Tv Libre Noticias']
['Venezuela es pasión']

0.0

0.5960986175547416
(73, 122)
573134508951-1549727034 / 573508650959-1572285131
['Tv Libre Noticias']
['CUCUTA^Paso^FRONTERA' 'CUCUTA^Paso^FRONTERA…']

0.0

0.66629096293576
(73, 126)
573134508951-1549727034 / 584124411924-1570375550
['Tv Libre Noticias']
['Venezuela y Colombia']

0.0

0.70666886


0.0

0.5404288269829839
(97, 148)
573204379940-1516889957 / 584245066057-1546970262
['Nacional de Cambios  .']
['CAMBIOS IP' 'Cambios IP']

0.0

0.6145590755487262
(97, 151)
573204379940-1516889957 / 584246249002-1565349721
['Nacional de Cambios  .']
['CASA DE CAMBIOS D&Y' 'CASA DE CAMBIOS D&Y…']

0.0

0.5199943388383801
(98, 154)
573204896999-1557775885 / 584247393733-1565574033
['>AMIGS SIN FRNTERAS<' '>AMIGS SIN FRNTERAS<…']
['GENTE CHEVERE']

0.0

0.5655496497658977
(101, 154)
573209056220-1582183315 / 584247393733-1565574033
['SalsaBaul ineditas']
['GENTE CHEVERE']

0.0

0.6797052452927306
(101, 161)
573209056220-1582183315 / 584264954187-1584036417
['SalsaBaul ineditas']
['Salsa baul.']

0.0

0.9106251233706617
(104, 167)
573212644564-1549511812 / 85700215ac095301c08be263a8e96d4a
['StarCambioS']
['StarCambioS']

MERGE

0.7176665064071458
(105, 118)
573213514329-1575474764 / 573227727724-1572791992
['El Mercadito Venezolano']
['Venezuela es pasión']

0.0617678381256656

0.5566777


0.0

1.0
(181, 189)
c156b079eb85b92d5b9d8ebe1d9b6a54 / f4002abec840067e43fb14e866247b5a
['Saʟsɛʀօs ɖɛ Cօʀaʐóռ']
['Saʟsɛʀօs ɖɛ Cօʀaʐóռ']

MERGE



In [15]:
print(len(rename_from_to))
print(df_x['uid'].unique().shape[0])

23
191


In [17]:
with open('Data/intermediate/rename_from_to.txt', 'w') as file:
    file.write(str(rename_from_to))

## Fix

In [18]:
df_x['uid'] = df_x['uid'].replace([i[0] for i in rename_from_to], [i[1] for i in rename_from_to])

# Remove duplicates again

In [19]:
# Duplicate messages within same traverse
intra_traverse_duplicates = df_x[df_x.duplicated(keep = False)]

# This gives us all duplicates that have the same timez.
# However, bona fide duplicates may be duplicated across multiple runs of traverse, so we only
# want one appearance of them.

In [20]:
# We can go through all possible bona fide duplicates
# and mark to keep them iff they are the first run bona fide duplicate

duplicates_first_appearance = []
i_true_duplicates = []
partials_collection = []

for i, row in intra_traverse_duplicates.iterrows():
    row_partial = row[['uid', 'event', 'text', 'emojis',
       'audio_length', 'image', 'reply_to', 'reply_text', 'reply_image',
       'reply_thumb', 'reply_emojis', 'reply_type_audio', 'reply_type_image',
       'message_deleted', 'video_length', 'video_thumb', 'forwarded',
       'forwarded_highly', 'tel', 'message_dt', 'message_date', 'textlower']]

    if str(row.values) in duplicates_first_appearance:
        i_true_duplicates.append(i)        
    else:
        if str(row_partial.values) in partials_collection:
            pass
        else:
            duplicates_first_appearance.append(str(row.values))
            partials_collection.append(str(row_partial.values))
            
print(len(i_true_duplicates))

1881


In [21]:
persist_cols = ['uid', 'title', 'event', 'text', 'emojis',
       'audio_length', 'image', 'reply_to', 'reply_text', 'reply_image',
       'reply_thumb', 'reply_emojis', 'reply_type_audio', 'reply_type_image',
       'message_deleted', 'video_length', 'video_thumb', 'forwarded',
       'forwarded_highly', 'tel', 'message_dt', 'message_date', 'textlower']

In [22]:
i_duplicates = df_x[df_x[persist_cols].duplicated(keep = 'first')].index.values

In [23]:
i_duplicates_remove = [i for i in i_duplicates if i not in i_true_duplicates]

In [24]:
print(len(i_duplicates))
print(len(i_duplicates_remove))

2542
661


In [25]:
df_x = df_x.drop(i_duplicates_remove, axis = 0)
print(df_x.shape)

(171634, 26)


In [26]:
df_x = df_x.reset_index(drop = True)

In [27]:
df_x.to_csv("Data/df_x_nb0a-groupMerge.csv")